# ETL – Data Preparation (Step 2)
## AI Impact on Job Market (2024–2030)

**Course:** IT300 – Tunis Business School  
**Phase:** Data Preparation & ETL  


In [13]:
# =====================================
# 1. IMPORT LIBRARIES
# =====================================
import pandas as pd
import numpy as np
import os


In [14]:
# =====================================
# 2. DEFINE PROJECT DIRECTORIES
# =====================================

BASE_DIR = ".."   # project root
RAW_DIR = os.path.join(BASE_DIR, "data", "raw")
STAGING_DIR = os.path.join(BASE_DIR, "data", "staging")
CLEANED_DIR = os.path.join(BASE_DIR, "data", "cleaned")

os.makedirs(STAGING_DIR, exist_ok=True)
os.makedirs(CLEANED_DIR, exist_ok=True)

print("Directories ready")


Directories ready


In [ ]:
# =====================================
# 3. EXTRACT – LOAD RAW DATA
# =====================================
import pandas as pd

raw_file_path = os.path.join(RAW_DIR, "ai_job_market.csv") 

df_raw = pd.read_csv(raw_file_path)

print("Raw data loaded successfully")
df_raw.head()


Raw data loaded successfully


,Job Title,Industry,Job Status,AI Impact Level,Median Salary (USD),Required Education,Experience Required (Years),Job Openings (2024),Projected Openings (2030),Remote Work Ratio (%),Automation Risk (%),Location,Gender Diversity (%)
0,Investment analyst,IT,Increasing,Moderate,42109.76,Master’s Degree,5,1515,6342,55.96,28.28,UK,44.63
1,"Journalist, newspaper",Manufacturing,Increasing,Moderate,132298.57,Master’s Degree,15,1243,6205,16.81,89.71,USA,66.39
2,Financial planner,Finance,Increasing,Low,143279.19,Bachelor’s Degree,4,3338,1154,91.82,72.97,Canada,41.13
3,Legal secretary,Healthcare,Increasing,High,97576.13,Associate Degree,15,7173,4060,1.89,99.94,Australia,65.76
4,Aeronautical engineer,IT,Increasing,Low,60956.63,Master’s Degree,13,5944,7396,53.76,37.65,Germany,72.57


In [ ]:
# =====================================
# 4. SAVE TO STAGING AREA 
# =====================================
staging_file_path = os.path.join(STAGING_DIR, "ai_job_market_staging.csv")

df_raw.to_csv(staging_file_path, index=False)

print("Data saved to staging area")


Data saved to staging area


In [ ]:
# =====================================
# 5. LOAD DATA FROM STAGING 
# =====================================
df = pd.read_csv(staging_file_path)

print("Data loaded from staging area")
df.info()  


Data loaded from staging area
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Job Title                    30000 non-null  object 
 1   Industry                     30000 non-null  object 
 2   Job Status                   30000 non-null  object 
 3   AI Impact Level              30000 non-null  object 
 4   Median Salary (USD)          30000 non-null  float64
 5   Required Education           30000 non-null  object 
 6   Experience Required (Years)  30000 non-null  int64  
 7   Job Openings (2024)          30000 non-null  int64  
 8   Projected Openings (2030)    30000 non-null  int64  
 9   Remote Work Ratio (%)        30000 non-null  float64
 10  Automation Risk (%)          30000 non-null  float64
 11  Location                     30000 non-null  object 
 12  Gender Diversity (%)         30000 non-null 

In [ ]:
# =====================================
# 6. DATA QUALITY ASSESSMENT
# =====================================

# Check missing values per column
missing_values = df.isnull().sum().sort_values(ascending=False)
print("Missing values per column:\n", missing_values)

# Check duplicate rows
duplicates = df.duplicated().sum()
print("Number of duplicate rows:", duplicates)

# Outlier Detection 
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

outliers_summary = {}

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outliers_summary[col] = outliers_count

print("Outliers detected using IQR method:")
for col, count in outliers_summary.items():
    print(f"{col}: {count} outliers")


Missing values per column:
 Job Title                      0
Industry                       0
Job Status                     0
AI Impact Level                0
Median Salary (USD)            0
Required Education             0
Experience Required (Years)    0
Job Openings (2024)            0
Projected Openings (2030)      0
Remote Work Ratio (%)          0
Automation Risk (%)            0
Location                       0
Gender Diversity (%)           0
dtype: int64
Number of duplicate rows: 0
Outliers detected using IQR method:
Median Salary (USD): 0 outliers
Experience Required (Years): 0 outliers
Job Openings (2024): 0 outliers
Projected Openings (2030): 0 outliers
Remote Work Ratio (%): 0 outliers
Automation Risk (%): 0 outliers
Gender Diversity (%): 0 outliers


In [ ]:
# =====================================
# 7.1 Standardize Text Columns
# =====================================
text_columns = df.select_dtypes(include='object').columns

for col in text_columns:
    df[col] = df[col].str.strip()  # remove leading/trailing spaces
    df[col] = df[col].str.title()  # Capitalize first letter of each word

print("Text columns standardized")


Text columns standardized


In [ ]:
# =====================================
# 7.2 Fix Percentage Columns
# =====================================
percentage_cols = ['Automation Risk (%)', 'Remote Work Ratio (%)']  

for col in percentage_cols:
    if col in df.columns:
        
        df[col] = df[col].astype(str).str.replace('%', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')  

print("Percentage columns fixed")


Percentage columns fixed


In [ ]:
# =====================================
# 7.3 Fix Salary & Numeric Columns
# =====================================
# Ensure all numeric columns are actually numeric
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  


for col in ['Job Openings 2024', 'Projected Openings 2030', 'Median Salary']:
    if col in df.columns:
        df[col] = df[col].clip(lower=0)  # set any negative values to 0

print("Numeric columns fixed")


Numeric columns fixed


In [22]:
# =====================================
# 8. DATA TRANSFORMATION – DERIVED METRICS
# =====================================

# Strip column names to avoid extra spaces
df.columns = df.columns.str.strip()

# ---- 8.1 Derived Metrics: Net Job Change and Percent Job Change ----
if 'Job Openings (2024)' in df.columns and 'Projected Openings (2030)' in df.columns:
    df['Net Job Change'] = df['Projected Openings (2030)'] - df['Job Openings (2024)']
    df['Percent Job Change'] = (df['Net Job Change'] / df['Job Openings (2024)']) * 100

# ---- 8.2 Encode AI Impact Level (Ordinal) ----
ai_impact_map = {'Low': 1, 'Moderate': 2, 'High': 3}
if 'AI Impact Level' in df.columns:
    df['AI Impact Score'] = df['AI Impact Level'].map(ai_impact_map)

# ---- 8.3 Career Stability Index ----
if 'Automation Risk (%)' in df.columns and 'Percent Job Change' in df.columns:
    df['Automation Risk (%)'] = pd.to_numeric(df['Automation Risk (%)'], errors='coerce')
    df['Career Stability Index'] = (1 - df['Automation Risk (%)'] / 100) * df['Percent Job Change']

# ---- 8.4 High-Opportunity Career Score ----
if 'Median Salary (USD)' in df.columns and 'Percent Job Change' in df.columns:
    df['Median Salary (USD)'] = pd.to_numeric(df['Median Salary (USD)'], errors='coerce')
    min_salary = df['Median Salary (USD)'].min()
    max_salary = df['Median Salary (USD)'].max()
    df['Median Salary Norm'] = (df['Median Salary (USD)'] - min_salary) / (max_salary - min_salary)
    df['High-Opportunity Career Score'] = (
        0.4 * df['Percent Job Change']/100 + 
        0.3 * df['Median Salary Norm'] + 
        0.3 * (1 - df['Automation Risk (%)']/100)
    )

# ---- 8.5 Round numeric metrics to 2 decimal places ----
columns_to_round = [
    'Percent Job Change',
    'Career Stability Index',
    'High-Opportunity Career Score',
    'Median Salary Norm'
]

for col in columns_to_round:
    if col in df.columns:
        df[col] = df[col].round(2)

print("Derived metrics created and rounded successfully")
df.head()

Derived metrics created and rounded successfully


,Job Title,Industry,Job Status,AI Impact Level,Median Salary (USD),Required Education,Experience Required (Years),Job Openings (2024),Projected Openings (2030),Remote Work Ratio (%),Automation Risk (%),Location,Gender Diversity (%),Net Job Change,Percent Job Change,AI Impact Score,Career Stability Index,Median Salary Norm,High-Opportunity Career Score
0,Investment Analyst,It,Increasing,Moderate,42109.76,Master’S Degree,5,1515,6342,55.96,28.28,Uk,44.63,4827,318.61,2,228.51,0.10,1.52
1,"Journalist, Newspaper",Manufacturing,Increasing,Moderate,132298.57,Master’S Degree,15,1243,6205,16.81,89.71,Usa,66.39,4962,399.20,2,41.08,0.85,1.88
2,Financial Planner,Finance,Increasing,Low,143279.19,Bachelor’S Degree,4,3338,1154,91.82,72.97,Canada,41.13,-2184,-65.43,1,-17.69,0.94,0.10
3,Legal Secretary,Healthcare,Increasing,High,97576.13,Associate Degree,15,7173,4060,1.89,99.94,Australia,65.76,-3113,-43.40,3,-0.03,0.56,-0.00
4,Aeronautical Engineer,It,Increasing,Low,60956.63,Master’S Degree,13,5944,7396,53.76,37.65,Germany,72.57,1452,24.43,1,15.23,0.26,0.36


In [24]:
# =====================================
# 9. LOAD – SAVE CLEAN DATASET
# =====================================

# Define the path for the cleaned CSV
cleaned_file_path = os.path.join(CLEANED_DIR, "ai_job_market_cleaned.csv")

# Save the DataFrame
df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved successfully at: {cleaned_file_path}")

Cleaned dataset saved successfully at: ..\data\cleaned\ai_job_market_cleaned.csv
